In [1]:
using BenchmarkTools: @btime
import LinearAlgebra as la

In [4]:
include("HF/hf.jl")
include("Basis/basis.jl")
include("Basis/pairing.jl")
include("Basis/HOBasis.jl")
include("system.jl")

# HF

In [7]:
l = 20
ω = 0.25
basis = HOBasis(l, ω)

n = 2
a = 0.25  # shielding term in coloumb interaction
grid = [x for x in range(-10, stop = 10, length = 2001)]
system = System(n, basis, grid, a);

state = setup_HF(system);

In [ ]:
@time HF_update!(state);

In [ ]:
E_HF(state)

## VMC

In [ ]:
dims = 1
n = 50

ham = HOpotential(HOshape, ω = 0.5)
wf = Slater(hf_state)
state = setup_vmc(wf, ham)

In [ ]:
metro = Metropolis(10, 100, 0.5)
#metro = Importance(10, 100, 0.1)

scheme = GradientDescent(lr = 0.01, maxiter = 10, tol = 1e-6)

optimize!(state, metro, scheme)

In [ ]:
blocking_energy(state, metro)

In [ ]:
l = 20
ω = 0.25
ho = HOBasis(l, ω)

n = 2
a = 0.25  # shielding term in coloumb interaction
grid = [x for x in range(-10, stop = 10, length = 2001)]
system = System(n, ho, grid, a);

In [ ]:
x = zeros(l);
x2 = zeros(2l);

In [ ]:
function spin!(x2, x, p, ho)
    fast_ho!(x, p, ho);
    
    @inbounds for i in eachindex(x)
        x2[2i-1] = x[i]
        x2[2i] = x[i]
    end
    return x2
end

In [ ]:
@btime spin!(x2, x, 3.0, ho);

In [ ]:
@btime fast_ho2!(x2, 3.0, ho);

In [ ]:
function fast_ho2!(hos, x, ho)
    (; ω, hermites) = ho
    
    x = √ω * x

    hermites[1] = 1.0
    hermites[2] = 2x

    ho_fac = (ω / π)^0.25 * exp(-x^2 / 2)
    hos[1] = ho_fac * hermites[1]
    hos[2] = ho_fac * hermites[1]
    ho_fac *= 1 / √2
    hos[3] = ho_fac * hermites[2]
    hos[4] = ho_fac * hermites[2]

    @inbounds for n in 3:ho.l
        hermites[n] = 2x * hermites[n-1] - 2(n - 2) * hermites[n-2]

        ho_fac *= 1 / sqrt( 2(n - 1) )
        hos[2n-1] = ho_fac * hermites[n]
        hos[2n] = ho_fac * hermites[n]
    end
    
    return hos
end

In [ ]:
struct A
    a::Vector{Float64}
end

function getResult(x, a)
    return a.a
end

b = A(rand(50));

function test(x, a)
    g = getResult(x, a)
    return g
end

In [ ]:
@time test(3.0, b);

In [10]:
include("VMC/WaveFunctions/slater.jl")

consider! (generic function with 1 method)

In [12]:
wf = Slater(state);